# 6 - Registering a Data file (as URL)

Demonstrates how to register a new Data file, in this case without upload but through a URL.

In [0]:
import requests
import json
import string

import getpass

In [0]:
base_url = 'https://sandbox3.fairdomhub.org'

Set up the headers and authenticate just as in the earlier steps.

In [0]:
headers = {"Content-type": "application/vnd.api+json",
           "Accept": "application/vnd.api+json",
           "Accept-Charset": "ISO-8859-1"}

session = requests.Session()
session.headers.update(headers)
session.auth = (input('Username:'), getpass.getpass('Password'))

The assay_id is the ID of the Assay determined in the earlier step

In [0]:
containing_project_id = 1
assay_id = 2

The general data_file JSON structure is built as a Hash, just as we did with Investigation, Study and Assay. In this case, the policy says that anybody can download.

The license is set as [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/). The list of available licenses can be found in the SEEK [API Overview](https://docs.seek4science.org/tech/api/index.html)

In [0]:
data_file = {}
data_file['data'] = {}
data_file['data']['type'] = 'data_files'
data_file['data']['attributes'] = {}
data_file['data']['attributes']['title'] = 'my lovely datafile'
data_file['data']['attributes']['license'] = 'CC-BY-4.0'
data_file['data']['attributes']['policy'] = {'access':'download'}
data_file['data']['relationships'] = {}
data_file['data']['relationships']['projects'] = {}
data_file['data']['relationships']['projects']['data'] = [{'id' : containing_project_id, 'type' : 'projects'}]

We describe a Content Blob just as it is was described in a earlier step. In this case we use the url key, and point to an available file (not webpage). In this case a copy of the FAIRDOM logo in our Github repository. A custom filename has also been provided, without this the filename would be determined from the link.

In [0]:
remote_blob = {'url' : 'https://github.com/seek4science/seek/raw/master/app/assets/images/logos/fairdom-logo.png', 'original_filename':'logo.png'}
data_file['data']['attributes']['content_blobs'] = [remote_blob]

Just as before, the data file and content blob is registered in one step with a POST to the data_files root.

In [0]:
r = session.post(base_url + '/data_files', json=data_file)
r.raise_for_status()

The resulting JSON should provde the details about the created Data file. Note that in the content blob details, the size and content_type have automatically been determined by inspecting the URL (a HEAD request is used).

In [0]:
populated_data_file = r.json()
populated_data_file

We now link to the Assay created earlier. We could have linked this in one step, just as we linked Investigations, Studies and Assays. However, in this case the Assay will be updated, to demonstrate updating an existing resource using a PATCH (or PUT) request.

Only the attributes that need changing need to be provided. In this case the title and data file links are changed. 

**Note** that if previous data files were linked, in this example the links would be replaced with data_file_id. In a real application you'd need to be careful to read and preserve existing links if desired.

In [0]:
data_file_id = populated_data_file['data']['id']
assay = {}
assay['data'] = {}
assay['data']['type'] = 'assays'
assay['data']['attributes'] = {}
assay['data']['attributes']['title'] = 'my lovely assay linked to data file'
assay['data']['relationships'] = {}
assay['data']['relationships']['data_files'] = {}
assay['data']['relationships']['data_files']['data'] = [{'id' : data_file_id, 'type' : 'data_files'}]

assay_url = base_url + '/assays/' + str(assay_id)

r = session.put(assay_url, json=assay)
r.raise_for_status()

assay_url

# Exercise 6



*  Find a URL to a resource online. This can be a picture on a website or a raw file in github. Update the data file to use that URL
*  Using the assay ID from the previous step, link the data to that assay and also update the title to something new. 


